In [1]:
import flywheel
import pandas as pd
fw = flywheel.Client()
project_id = '5c508d5fc2a4ad002d7628d8'
gear = fw.lookup('gears/antsct-aging-fw')

In [2]:
# Here we get all the sessions within the collection.
# We use `get_session` to return the associated analyses
all_sessions = [fw.get_session(x.id) for x in fw.get_project_sessions(project_id)]

In [3]:
df = pd.read_excel('/home/will/Projects/PVS/PVS_subjects.xlsx')
inddid_list = list(df['INDDID'])
inddid_list = [str(x) for x in inddid_list]
df.head()

,INDDID,MRIDate,AgeatMRI,MRIProtocol,Scanner,PETDate,PETTracer,Clinical_Read
0,122975,2021-04-08,72,NACC-SC,SC (3T),2021-04-13,Florbetaben (amyloid),Negative
1,125572,2021-03-23,63,NACC-SC,SC (3T),2019-11-05,Florbetaben (amyloid),Negative
2,124568,2021-03-18,70,NACC-SC,SC (3T),2019-05-09,Florbetapir (amyloid),Negative
3,119254,2021-03-11,73,NACC-SC,SC (3T),2020-12-22,Florbetaben (amyloid),Negative
4,101162,2021-01-28,65,NACC-SC,SC (3T),2020-11-19,Florbetaben (amyloid),Negative


In [4]:
print(inddid_list)

['122975', '125572', '124568', '119254', '101162', '124822', '123831', '124775', '100049', '125107', '123575', '118743', '107486', '118998', '124509', '120686', '112286', '124048', '118317', '124597', '125699', '124534', '124563', '125067', '122216', '119582', '121847', '123865', '111768', '117158', '118418', '124877', '100905', '118775', '122039', '122963', '122328', '123754', '120123', '123782', '107799', '122407', '118711', '116638', '117326', '119473', '123768', '124516', '124288', '103907', '124868', '101066', '125111', '124800', '124972', '106722', '123903', '118923', '115050', '124738', '124212', '123367', '123485', '113066', '118601', '119035', '122865', '126004', '125840', '122139', '125675', '125800', '121526', '124305', '111768', '114699', '121570', '120735', '124022', '120870', '117889', '108672', '105732', '105733', '101628', '117796', '121674', '121444', '106721', '119655', '118068', '119888', '116630', '121524', '124005', '119851', '124592', '122987', '123194', '109459',

In [5]:
sessions = []
for ses in all_sessions:
    sub = fw.get(ses.parents['subject'])
    if any(sub.label in string for string in inddid_list) and '3T' in ses.label:
        sessions.append(ses)

print(len(inddid_list))
print(len(sessions))

132
178


In [6]:
def run_gear(sessions, label):
    """
    Execute a series of Flywheel gear runs based on provided sessions.
    
    sessions: list of Flywheel session objects
    label: string of label ID
    
    """
    
    input_t1s = []
    no_t1_sessions = []
    analysis_ids = []
    for session in sessions:
        ## Get acquisition
        potential_t1s = []
        sub = fw.get(session.parents['subject'])
        for acq in session.acquisitions():
            class_t1 = False
            class_struct_intent = False
            class_mprage = False
            if acq.files and len(acq.files) > 0:
                for i in range(0, len(acq.files)):
                    try: 
                        class_t1 = 'T1' in acq.files[i]['classification']['Measurement'] 
                        class_struct_intent = 'Structural' in acq.files[i]['classification']['Intent']
                        class_mprage = 'MPRAGE' in acq.files[i]['classification']['Features']
                    except TypeError as te:
                        continue
                    except KeyError as ke:
                        continue                
                    break                    

            # follow a hierarchy of surest identifiers of the desired T1 acquisition (usually mprage, never setter, etc.)
            is_mprage = class_mprage or "mprage" in acq.label.lower() or 't1w_mpr' in acq.label.lower()                           
            if is_mprage and "setter" not in acq.label:
                potential_t1s.insert(0, acq)
            elif class_t1 and class_struct_intent and "setter" not in acq.label:
                potential_t1s.append(acq)
            else:
                pass

        if len(potential_t1s) == 0: # set aside if no T1s are found
            no_t1_sessions.append('{}: {}'.format(sub.label, session.label))
        else:
            # add the highest priority acquisition to the input T1s list
            input_t1_acq = potential_t1s[0]
            input_t1s.append(input_t1_acq)

        for f in input_t1_acq.files:
            if 'nii.gz' in f.name:
                input_file = f
                break
        
        print('{}: {}'.format(sub.label, input_t1_acq.label))

        # Run the gear
        inputs = {'t1_anatomy': input_file}
        config = {'denoise': True, 'num-threads': 0, 'trim-neck-mode': 'mask', 'run-quick': False}
        analysis_id = gear.run(analysis_label=label, config=config, inputs=inputs, destination=session)
        analysis_ids.append(analysis_id)
        
    return input_t1s, no_t1_sessions, analysis_ids

In [7]:
# do a test/run the gear
test_sessions = sessions[0:3]
(input_t1s, no_t1_sessions, analysis_ids) = run_gear(sessions, 'antsct_2021-05-13_WT_PVS')

123572: T1_3D_0.8x0.8x0.8_ND
123725: T1_3D_0.8x0.8x0.8_ND
122475: T1_3D_0.8x0.8x0.8_ND
123894: T1_3D_0.8x0.8x0.8_ND
109459: T1_3D_0.8x0.8x0.8_ND
120896: T1_3D_0.8x0.8x0.8_ND
115562: T1_3D_0.8x0.8x0.8_ND
119035: T1_3D_0.8x0.8x0.8
118601: T1_3D_0.8x0.8x0.8
118711: T1_3D_0.8x0.8x0.8
121524: T1_3D_0.8x0.8x0.8
121674: T1_3D_0.8x0.8x0.8
123367: T1_3D_0.8x0.8x0.8_ND
119851: T1_3D_0.8x0.8x0.8
121570: T1_3D_0.8x0.8x0.8_ND
123194: T1_3D_0.8x0.8x0.8
119851: T1_3D_0.8x0.8x0.8
116638: T1_3D_0.8x0.8x0.8
119888: T1_3D_0.8x0.8x0.8
122039: T1_3D_0.8x0.8x0.8
119254: T1_3D_0.8x0.8x0.8_ND
123485: T1_3D_0.8x0.8x0.8
107799: T1_3D_0.8x0.8x0.8
115050: T1_3D_0.8x0.8x0.8
117326: T1_3D_0.8x0.8x0.8
120896: T1_3D_0.8x0.8x0.8
116707: T1_3D_0.8x0.8x0.8
120686: T1_3D_0.8x0.8x0.8
101162: T1_3D_0.8x0.8x0.8_ND
120965: T1_3D_0.8x0.8x0.8
118775: T1_3D_0.8x0.8x0.8
117358: T1_3D_0.8x0.8x0.8
101453: T1_3D_0.8x0.8x0.8_ND
120515: T1_3D_0.8x0.8x0.8_ND
101162: T1_3D_0.8x0.8x0.8
118317: T1_3D_0.8x0.8x0.8
115562: T1_3D_0.8x0.8x0.8

In [8]:
for s in test_sessions:
    print(s.label)

123572x20190313x3T
123725x20190327x3T
122475x20190402x3T


### Add sessions to collection

In [9]:
collection = fw.get('60a4032befa949862afeafbb')
for ses in sessions:
    print(ses.label)
    #collection.add_sessions(ses)

123572x20190313x3T
123725x20190327x3T
122475x20190402x3T
123894x20190327x3T
109495x20190417x3T
120896x20190409x3T
115562x20190409x3T
119035x20190521x3T
118601x20190523x3T
118711x20160412x3T
121524x20170620x3T
121674x20170815x3T
123367x20190529x3T
119851x20170606x3T
121570x20171219x3T
123194x20190603x3T
119851x20190613x3T
116638x20170606x3T
119888x20170413x3T
122039x20170905x3T
119254x20181204x3T
123485x20190529x3T
107799x20160804x3T
115050x20170523x3T
117326x20170926x3T
120896x20170330x3T
116707x20160620x3T
120686x20160414x3T
101162x20181017x3T
120965x20180821x3T
118775x20160523x3T
117358x20170202x3T
101453x20181016x3T
120515x20181127x3T
101162x20160523x3T
118317x20160915x3T
115562x20170309x3T
118700x20181009x3T
118998x20160901x3T
119254x20160428x3T
109459x20170413x3T
122407x20171017x3T
124212x20190604x3T
124592x20190611x3T
119473x20170105x3T
119655x20170504x3T
100049x20160621x3T
101066x20170601x3T
103907x20170202x3T
120735x20171221x3T
118700x20160509x3T
117158x20170613x3T
118418x20170

In [10]:
ses_list = fw.get_collection_sessions('60a4032befa949862afeafbb')
print(len(ses_list))

178
